In [ ]:
# in case you dont have the packages installed, 
# use these for installation in an R kernel for jupyter notebook

#install.packages("tidyr")
#install.packages("ggfortify")
#install.packages("ggrepel")
#install.packages("BiocManager")
#install.packages("WebGestaltR")
#Sys.setenv(R_INSTALL_STAGED = FALSE)
#BiocManager::install("DESeq2")
#BiocManager::install("cowplot")

In [ ]:
# load libraries required in this notebook

library(tidyr)
library(DESeq2)
library(ggplot2)
library(reshape2)
library(dplyr)
library(Rtsne)
library(caret)
library(ggfortify)
library(ggrepel)
library(WebGestaltR)
library(RCurl)

In [ ]:
# definition of function for plotting PCA colored with different metadata
# needed later
preparePCAplots = function(pca, metadata_of_interest){

    data_to_plot <- cbind(data.frame(PC1 = pca$x[,1], PC2 = pca$x[,2]), metadata_of_interest)
    features_to_plot <- setdiff(colnames(data_to_plot), c("PC1", "PC2"))
    features_to_plot <- setNames(features_to_plot, features_to_plot)
    all_plots_pca <- lapply(features_to_plot,
           function(col) {
                        gg = ggplot(data = data_to_plot) +
                        geom_point(aes(x = PC1, y = PC2, color = !!sym(col)), size = 5) +
                        theme(legend.position = "bottom", legend.margin=margin(t=0, unit="pt"),  
                              text = element_text(size=14), 
                              legend.title = element_text(size = 10),
                        legend.text = element_text(size = 10)) + xlab("") + ylab("") # xlim(-5,5) + ylim(-5,5)+ 
                        if (class(data_to_plot[,col]) == "factor")
                        gg = gg + guides(colour = guide_legend(title.position = "top", nrow = 2))
                        if (class(data_to_plot[,col]) == "character")
                        gg = gg + guides(colour = guide_legend(title.position = "top", nrow = 3))
                          
           return(gg)}
    )
    return(all_plots_pca)
}

In [ ]:
# define where Pathonoia output is stored. or example generated in with 
# Pathonoia/pathonoia_notebook.ipynb 
# it is the input for this downstream analysis

inputfolder = "~/Pathonoia/OUTPUT/"

# subfolder in OUTPUT folder in case you have more than one dataset
ds = "FTD"

# Loading and Processing of Data and Metadata

In [ ]:
# You might define outliers later which will be ignored 
# when loading the data.
# For now define empty variable

pca_outlier = c()

In [ ]:
# read in Pathonoia Abundance file
counts = read.table(paste0(inputfolder,ds,"_contamination_aggregated.csv"), sep = ",", header=TRUE, check.names = FALSE, row.names = "")

# remove Outliers if defined
# print Outlier sample names if defined
if(exists("pca_outlier")){
    print(pca_outlier)
    counts = counts[,!(colnames(counts) %in% pca_outlier)]
} else {
    pca_outlier = c()
}

# split out information columns from the count data 
# and store in seperate variable
pathoInfoCols = c("species_name","phylo_level","parent")
pathoInfo = counts[pathoInfoCols]
counts = counts[,!colnames(counts) %in% pathoInfoCols]

# set NA values in count matrix to zero
counts[is.na(counts)]=0

# display matrix which should have sample names in columns
# and taxonomic IDs in row names
counts

In [ ]:
# If outliers were removed, zero rows might have been introduced. 
# They'll be removed with this code
# Additionally were sparse rows can be removed (adjustable)

noOfOutliers = length(pca_outlier)
numberOfZerosAllowed = length(counts)*0.99
counts_lessSparse = counts[!(rowSums(counts == 0) > numberOfZerosAllowed),]
table(duplicated(counts_lessSparse))
counts = counts_lessSparse

In [ ]:
# load the metadata from file
metadata = read.table(paste0(inputfolder,"metadata.csv"))
# set rownames with sample IDs used in count matrix.
# this might need adjustments, adjust accordingly
# e.g. 
rownames(metadata) = metadata$sample
# or
# rownames(metadata) = substr(rownames(metadata),1,5)

# align metadata with count file
metadata = metadata[colnames(counts),]

# look at your own metadata variables, 
# do you want to analise the PCA with all columns?
colnames(metadata)

# get insights to your metadata, for example with
table(metadata$CASE.CONTROL)

# you may want to do some cleaning of your metadata, 
# for example like this
# metadata$fibrosis = gsub('0-1', '0.5', metadata$fibrosis)
# metadata$PH = as.numeric(metadata$PH)

# look at your metadata
metadata

In [ ]:
# select your comuns  (of metadata) of interest, for example
cols_of_interest = c("sex",'age','diagnosis','fibrosis','mHAI')

metadata_of_interest = metadata[rownames(t(counts)),cols_of_interest]
metadata_of_interest

In [ ]:
# COME BACK TO THIS CELL LATER, 
# when you have identified an Organism of Interest

# add the abundance data of one specific organism (tax ID) 
# to the metadata of interest for visualization in PCR

metadata_of_interest = cbind(metadata_of_interest,t(counts["95485",rownames(metadata_of_interest)]))
metadata_of_interest$"95485" = as.factor(metadata_of_interest$"95485")
metadata_of_interest

# PCA analysis

In [ ]:
#calculate PCA from counts
pca <- prcomp(t(counts), scale = TRUE)

#adjust size of your plot
options(repr.plot.width=8, repr.plot.height=6)

# plot your PCA
# set label = TRUE if you want to see sample names (for Outlier identification)
autoplot(pca, label = FALSE, size = 4, label.vjust = 1.5) + theme(text = element_text(size=18))


In [ ]:
# define Outliers which you would like to remove from the analysis
pca_outlier = c('samplename1','sample34','s67')

# if you wish to reset:
#pca_outlier = c()

# RUN CELLS AGAIN FROM ABOVE loading the data

In [ ]:
# prepare plots for coloring with metadata
all_plots_pca = preparePCAplots(pca,metadata_of_interest)

# set plot size
options(repr.plot.width=20, repr.plot.height=40)

# plot all with metadata of interest
cowplot::plot_grid(plotlist = all_plots_pca, ncol = 4, align = "h") 

# Differential Expression

In [ ]:
# define the two groups that you want to compare, for example
metadata$DE_GROUP = metadata$`CASE.CONTROL`
# or
# metadata$DE_GROUP = metadata$fibrosis < 1

In [ ]:
# run DESeq2 on abundance counts

dds <- DESeqDataSetFromMatrix(countData = counts[,rownames(metadata)],
                              colData = metadata,
                              design= ~ CASE.CONTROL)
dds <- DESeq(dds)
resultsNames(dds) 

In [ ]:
# add taxonomy information to differential abundance results
# filter for significant results only (adjusted p-value < 0.05)
# sort by p-value and display results

deres = as.data.frame(results(dds, name=resultsNames(dds)[2]))
r = cbind(pathoInfo[rownames(deres),pathoInfoCols],deres)
r = r[r$padj<0.05 & !is.na(r$padj),]
r = r[order(r$padj),]
r

In [ ]:
# plot abundance counts of 12 (adjust as desired) most significant organisms
# define (in code line 6) metadata column according to which 
# the counts should be split per organism sub-plot 

topX = 12
samples = rownames(metadata_of_interest)
t = counts[rownames(r[1:topX,]),samples] 
t$organism = r[1:topX,"species_name"]
t = gather(t, key = "sample", value = "abundance", -organism)

# replace 'fibrosis' with your metadata column of interest.
t = cbind(t, cc = metadata_of_interest[t$sample,"fibrosis"])
# or
# t = cbind(t, cc = metadata[t$sample,"DE_GROUP"])

# adjust plot size
options(repr.plot.width=18, repr.plot.height=12)

# plotting function for all top organisms, change xlabel in last line as desired
ggplot(t, aes(x = cc, y = abundance)) + theme_minimal() + 
    geom_jitter(width = 0.15, size=5) +
    stat_summary(fun=mean, geom="point", shape=95, size=18, color="blue") + 
    stat_summary(fun=median, geom="point", shape=95, size=18, color="red") +
    theme(text = element_text(size=14), strip.text = element_text(size=18))+ 
    facet_wrap(facets = "organism", scales = "free", ncol=3) + 
    ggtitle(paste("red = median, blue = mean")) + xlab("your X label") + ylab("Pathonoia Abundance")


In [ ]:
# Volcano Plot including coloring of most present organisms

r = cbind(r, posi_samples = rowSums(counts[rownames(r),]>0))
r$log10_padj=-log10(r$padj)

# adjust the label_cutoff according to your dataset size
# it only shows the names of the most present organisms
label_cutoff = 9

# plotsize
options(repr.plot.width=9, repr.plot.height=6)

# plotting function
ggplot(r, aes(log2FoldChange, log10_padj, color= posi_samples, label=ifelse(posi_samples>label_cutoff, as.character(species_name),""))) + 
geom_point(size=3) + theme_minimal() + geom_text_repel(min.segment.length = 0) 


In [ ]:
# NOW CHOOSE YOUR ORGANISM OF INTEREST
# is it more than one? 
# run this several times or add several where appropriate

# in this notebook we keep working with the taxID, not the species name
# since they are the rownames in the count data
# the link between tax ID and name can be (for example) found in pathoInfo

taxID_OOI = "95485"

# Scroll up to the PCA analysis and color it according to this taxID

## DE Transcriptome

here, we try to find evidence for the organisms of interest having influence on a sample group|

In [ ]:
# if you have RNAseq data for the same samples, get the gene count matrix, 
# for example derived with STAR

genecounts = read.table(paste0(inputfolder,"/RNAseq_gene_count_summary.txt"), header = TRUE, row.names = 1)
# only use those samples which were used in the previous analysis
genecounts = genecounts[,samples]
genecounts

In [ ]:
# prep for Differential Expression Analysis 
# using taxID of organism of interest
metadata$GENE_DE =  as.factor(t(counts[taxID_OOI,rownames(metadata)] > 0)) # 0 genes

# define a subgroup of samples, for example "only patients" 
# using the same metadata from before
samples_gde = rownames(metadata[metadata$fibrosis >= 1,])
metadata_gde = metadata[samples_gde,]
genecounts = genecounts[,samples_gde]

In [ ]:
# Run DESeq2 for differential gene expression analysis
dds_g <- DESeqDataSetFromMatrix(countData = genecounts,
                              colData = metadata_gde,
                              design= ~ GENE_DE)
dds_g <- DESeq(dds_g)
resultsNames(dds_g) 

In [ ]:
# display only significant DE genes

gderes = as.data.frame(results(dds_g, name=resultsNames(dds_g)[2]))
gr = gderes[gderes$padj<0.05 & !is.na(gderes$padj),]
gr

# if this table is EMPTY, you might try different cut offs OR realize, 
# that there is absence of evidence that this organism 
# did anything to the sample group

In [ ]:
# make sets of up-, down- regulated genes

upregulated_genes = sapply(strsplit(rownames(gr[gr$log2FoldChange > 0,]), ".", fixed=T), function(x) x[1])
downregulated_genes = sapply(strsplit(rownames(gr[gr$log2FoldChange < 0,]), ".", fixed=T), function(x) x[1])
regulated_genes = sapply(strsplit(rownames(gr), ".", fixed=T), function(x) x[1])

# display genes downregulated (adjust with other var names) 
# by the organism of interest in the selected sample group
downregulated_genes

## Gene Ontology Analysis with WebGestalt

In case you some dis-regulated genes could be identified, a gene ontology analysis can be executed with the following code

In [ ]:
# this cell is setting up the analysis with WebGestalt
# you may want to adjust some parameters, for example
# interestGeneType="ensembl_gene_id"
# change this according to the gene identifiers that were used 
# in your gene count matrix

# make sure the folder "GO_results" exists in your working directory 
# or change it accordingly

WebGes_BioProc <- function(genes, projectname){
    WebGestaltR(enrichMethod="ORA", organism="hsapiens",
    enrichDatabase="geneontology_Biological_Process",
    interestGene=genes,interestGeneType="ensembl_gene_id",referenceGeneType="genesymbol",
    referenceSet="genome_protein-coding", minNum=5, maxNum=2000,
    fdrMethod="BH",sigMethod="fdr",fdrThr=0.05,topThr=10,reportNum=20,perNum=1000,
    nThreads=64,
    isOutput=TRUE,outputDirectory="GO_results/GO_Analysis_biolProcess_FDR05",projectName=projectname,
    dagColor="continuous",hostName="http://www.webgestalt.org/")
}

WebGes_MolFun <- function(genes, projectname){
    WebGestaltR(enrichMethod="ORA", organism="hsapiens",
    enrichDatabase="geneontology_Molecular_Function",
    interestGene=genes,interestGeneType="ensembl_gene_id",referenceGeneType="genesymbol",
    referenceSet="genome_protein-coding", minNum=5, maxNum=2000,
    fdrMethod="BH",sigMethod="fdr",fdrThr=0.05,topThr=10,reportNum=20,perNum=1000,
    nThreads=64,
    isOutput=TRUE,outputDirectory="GO_results/GO_Analysis_molFunction_FDR05",projectName=projectname,
    dagColor="continuous",hostName="http://www.webgestalt.org/")
}



In [ ]:
# create folder names and run WebGestalt for all sets

projectname = paste0(ds, "_", taxID_OOI, "_")

WebGes_BioProc(upregulated_genes, paste0(projectname,"upDEgenes"))
WebGes_BioProc(downregulated_genes, paste0(projectname,"downDEgenes"))
WebGes_BioProc(regulated_genes, paste0(projectname,"DEgenes"))
WebGes_MolFun(upregulated_genes, paste0(projectname,"_upDEgenes"))
WebGes_MolFun(downregulated_genes, paste0(projectname,"downDEgenes"))
WebGes_MolFun(regulated_genes, paste0(projectname,"DEgenes"))

# explore the results in the GO_results folder. 
# for example in the containing HTML file